In [2]:
import mysql.connector as connector
connection = connector.connect(user="root",password="",host="127.0.0.1")
cursor = connection.cursor()
cursor.execute("USE little_lemon_db;")

In [23]:
def action(sql):
    cursor.execute(sql)
    col = cursor.column_names
    results = cursor.fetchall()
    print(col)
    for item in results:
        print(item)

In [4]:
action("SELECT HOUR(BookingSlot) AS BookingTime FROM bookings")

('BookingTime',)
[(19,), (19,), (15,), (17,), (18,), (20,)]


In [8]:
action("SELECT HOUR(BookingSlot) AS BookingHour,COUNT(HOUR(BookingSlot)) AS NumOfBooking FROM bookings GROUP BY BookingHour ORDER BY NumOfBooking DESC  ")

('BookingHour', 'NumOfBooking')
[(19, 2), (15, 1), (17, 1), (18, 1), (20, 1)]


In [9]:
action("SELECT * FROM bookings");

('BookingID', 'TableNo', 'GuestFirstName', 'GuestLastName', 'BookingSlot', 'EmployeeID')
[(1, 12, 'Anna', 'Iversen', datetime.timedelta(seconds=68400), 1), (2, 12, 'Joakim', 'Iversen', datetime.timedelta(seconds=68400), 1), (3, 19, 'Vanessa', 'McCarthy', datetime.timedelta(seconds=54000), 3), (4, 15, 'Marcos', 'Romero', datetime.timedelta(seconds=63000), 4), (5, 5, 'Hiroki', 'Yamane', datetime.timedelta(seconds=66600), 2), (6, 8, 'Diana', 'Pinto', datetime.timedelta(seconds=72000), 5)]


In [11]:
first_procedure = """CREATE PROCEDURE PeakHours()
BEGIN
  SELECT HOUR(BookingSlot) AS BookingHour,COUNT(HOUR(BookingSlot)) AS NumOfBooking 
  FROM bookings 
  GROUP BY BookingHour 
  ORDER BY NumOfBooking 
  DESC;
END
"""
print(first_procedure)

CREATE PROCEDURE PeakHours()
BEGIN
  SELECT HOUR(BookingSlot) AS BookingHour,COUNT(HOUR(BookingSlot)) AS NumOfBooking 
  FROM bookings 
  GROUP BY BookingHour 
  ORDER BY NumOfBooking 
  DESC;
END



In [12]:
cursor.execute(first_procedure)

In [13]:
cursor.callproc("PeakHours")

()

In [15]:
results = next(cursor.stored_results())

In [16]:
col = results.column_names
data = results.fetchall()

In [17]:
print(col)
for item in data:
    print(item)

('BookingHour', 'NumOfBooking')
(19, 2)
(17, 1)
(18, 1)
(20, 1)
(15, 1)


In [25]:
action("SELECT * FROM Bookings")

('BookingID', 'TableNo', 'GuestFirstName', 'GuestLastName', 'BookingSlot', 'EmployeeID')
(1, 12, 'Anna', 'Iversen', datetime.timedelta(seconds=68400), 1)
(2, 12, 'Joakim', 'Iversen', datetime.timedelta(seconds=68400), 1)
(3, 19, 'Vanessa', 'McCarthy', datetime.timedelta(seconds=54000), 3)
(4, 15, 'Marcos', 'Romero', datetime.timedelta(seconds=63000), 4)
(5, 5, 'Hiroki', 'Yamane', datetime.timedelta(seconds=66600), 2)
(6, 8, 'Diana', 'Pinto', datetime.timedelta(seconds=72000), 5)


In [32]:
action("""SELECT CONCAT(B.GuestFirstName,' ',B.GuestLastName) AS GustName,
          CASE
              WHEN E.Role = "Manager" THEN "Ready to pay"
              WHEN E.Role = "Assistant Manager" THEN "Ready to pay"
              WHEN E.Role = "Head Chef" THEN "Ready to serve"
              WHEN E.Role = "Assistant Chef" THEN "Preparing Order"
              WHEN E.Role = "Head Waiter" THEN "Order served"
            END AS Status
          FROM bookings B LEFT JOIN Employees E 
          ON
          B.EmployeeID = E.EmployeeID;
          """)

('GustName', 'Status')
('Anna Iversen', 'Ready to pay')
('Joakim Iversen', 'Ready to pay')
('Vanessa McCarthy', 'Ready to serve')
('Marcos Romero', 'Preparing Order')
('Hiroki Yamane', 'Ready to pay')
('Diana Pinto', 'Order served')


In [33]:
action("SELECT COUNT(*) FROM Employees")

('COUNT(*)',)
(6,)


In [31]:
action("SELECT DISTINCT Role FROM Employees")

('Role',)
('Manager',)
('Assistant Manager',)
('Head Chef',)
('Assistant Chef',)
('Head Waiter',)
('Receptionist',)


In [35]:
second_procedure = """CREATE PROCEDURE GuestStatus()
BEGIN
    SELECT CONCAT(B.GuestFirstName,' ',B.GuestLastName) AS GustName,
          CASE
              WHEN E.Role = "Manager" THEN "Ready to pay"
              WHEN E.Role = "Assistant Manager" THEN "Ready to pay"
              WHEN E.Role = "Head Chef" THEN "Ready to serve"
              WHEN E.Role = "Assistant Chef" THEN "Preparing Order"
              WHEN E.Role = "Head Waiter" THEN "Order served"
          END AS Status
    FROM bookings B LEFT JOIN Employees E 
    ON
    B.EmployeeID = E.EmployeeID;
END
"""
print(second_procedure)

CREATE PROCEDURE GuestStatus()
BEGIN
    SELECT CONCAT(B.GuestFirstName,' ',B.GuestLastName) AS GustName,
          CASE
              WHEN E.Role = "Manager" THEN "Ready to pay"
              WHEN E.Role = "Assistant Manager" THEN "Ready to pay"
              WHEN E.Role = "Head Chef" THEN "Ready to serve"
              WHEN E.Role = "Assistant Chef" THEN "Preparing Order"
              WHEN E.Role = "Head Waiter" THEN "Order served"
          END AS Status
    FROM bookings B LEFT JOIN Employees E 
    ON
    B.EmployeeID = E.EmployeeID;
END



In [36]:
cursor.execute(second_procedure)

In [37]:
cursor.callproc("GuestStatus")

()

In [38]:
results = next(cursor.stored_results())

In [39]:
col = results.column_names
data = results.fetchall()
print(col)
for item in data:
    print(item)

('GustName', 'Status')
('Anna Iversen', 'Ready to pay')
('Joakim Iversen', 'Ready to pay')
('Vanessa McCarthy', 'Ready to serve')
('Marcos Romero', 'Preparing Order')
('Hiroki Yamane', 'Ready to pay')
('Diana Pinto', 'Order served')


In [40]:
action("DROP PROCEDURE PeakHours")
action("DROP PROCEDURE GuestStatus")

()
()
